# OceanOptics 2017
# Lab 5: Extracted Chlorophyll-a 


The purpose of this notebook is to calculate chlorophyll-a and pheophyton from our extracted chlorophyll-a samples that we have extracted. We will also propagate instrument uncertainty, compute the standard deviation of the replicates, and compare results bewrtwqeen students. Finally, we will calculate chlorophyll-a specific absorption coefficients for DRE samples. 

In [1]:
# This notebook was built in Catherine's root environment. (Sorry Don!) 
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import glob
import datetime
import pylab
import matplotlib as plt
import datetime as datetime  
import statsmodels.api as sm  
import matplotlib.pyplot as plt
import seaborn as sns

/Users/ckuhn/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
%matplotlib inline

### Import the data

In [3]:
%cd /Users/ckuhn/Documents/oceanoptics/Labs/Lab05/ # set to your working directory

[Errno 2] No such file or directory: '/Users/ckuhn/Documents/oceanoptics/Labs/Lab05/ # set to your working directory'
/Users/ckuhn/Documents/oceanoptics/Labs/Lab05-ap


This .csv contains the chl a and pheo a already calculated from the spreadsheet that was given to use by our amazing TA Jordan

In [4]:
data = pd.read_csv('test.csv', low_memory=False)

In [5]:
% pwd

u'/Users/ckuhn/Documents/oceanoptics/Labs/Lab05-ap'

# Set your global variables

In [31]:
# Uncertainty terms:
    
# sigma K given in the lab

sigma_K = 0.03
    
# These numbers are the uncertainty associated with when the instrument read the chla and pheo during calibration. 
# We are ballparking these numbers

sigma_phi_chla = 3
sigma_phi_pheo = 1.5

phi_chla = 500  # these are the conc associated with the standards 
phi_pheo = 250  # these are the conc associated with the standards
    
# These numbers are the uncertainty we observed when we were taking measurements. The initial chlorophyll and final 
# chlorophyll 

sigma_F_init  = 3
sigma_F_final = 1.5
       
# These are the calibration coefficients that we pulled from the lab spreadsheet from the last calibration, which was 
# conducted on April 27, 2012
    
K = 0.36 
A = 1.88

# Volumes

sigma_v = 0.0005   # let's say we spilled 0.5 mL
sigma_V = 0.005   # let's say we spilled 5 mL

v = 0.005  # acetone
V = 0.289  # volume of water filtered for the sample  

### Add in your sample information

In [32]:
data.head()

,Fo_FSU,Fa_FSU,Chl_ugL,Phaeo_ugL
0,590,358,3.07,1.97
1,573,345,3.02,2.35
2,536,330,2.72,2.58
3,638,389,3.29,2.87
4,609,363,3.25,2.30


In [34]:
varNo = 0 # change this to your sample number!

f_init = data['Fo_FSU'][varNo]
f_acid = data['Fa_FSU'][varNo]
chla_ugL = data['Chl_ugL'][varNo]
pheo_ugL = data['Phaeo_ugL'][varNo]

In [54]:
print 'initial F = ', f_init
print 'acid F =', f_acid
print 'chla_ug/L =', chla_ugL
print 'pheo_ug/L =', pheo_ugL

initial F =  590
acid F = 358
chla_ug/L = 3.07
pheo_ug/L = 1.97


Compute the uncertainty in your acid ratio (A)

In [53]:
sigma_A = 1/A * np.sqrt(((sigma_phi_chla/phi_chla)**2) + ((sigma_phi_pheo/phi_pheo)**2))
print 'sigma_A=', sigma_A

sigma_A= 0.0031914893617


Compute the uncertainty in your sigma A/A-1

In [37]:
sigma_AoverAminus1 = 1/ (A/(A-1)) * np.sqrt (((sigma_A/A)**2) + ((sigma_A/A)**2))
print(sigma_AoverAminus1)

0.00112376466578


Uncertainty in the flourescence readings

In [38]:
sigma_Finit_minus_Ffinal = np.sqrt (((sigma_F_init)**2) + ((sigma_F_final)**2))
print(sigma_Finit_minus_Ffinal)

3.35410196625


In [39]:
sigma_vV = sigma_v/sigma_V
print(sigma_vV)

0.1


In [40]:
sigma_chla = 1/chla_ugL * np.sqrt (((sigma_K/K)**2) + (sigma_AoverAminus1/ (A/(A-1))**2)
                                         + (sigma_Finit_minus_Ffinal/(f_init-f_acid)**2) 
                                         + ((sigma_vV/(v/V))**2))

## What the fuck is our uncertainty?

In [41]:
print(sigma_A)  # from calibration
print(sigma_K)   # more calibration error
print(sigma_vV) # from measuring water volume
print (sigma_Finit_minus_Ffinal) # uncertainty in our lab readings

print 'final error:', sigma_chla # total problems

0.0031914893617
0.03
0.1
3.35410196625
final error: 1.88294051677


In [ ]:
data['total_error_chla'] = sigma_chla

In [46]:
#errors = pd.Dataframe(sigma_A,sigma_K, sigma_vV, sigma_Finit_minus_Ffinal)
my_list =  [sigma_A,sigma_K, sigma_vV, sigma_Finit_minus_Ffinal]

In [48]:
df = pd.DataFrame(np.array(my_list), columns = list("abc"))

,0
0,0.003191
1,0.030000
2,0.100000
3,3.354102


In [ ]:
# One formula to rule them all 

For every row....

# Extract measurements from sheet
f_init = data['Fo_FSU'][varNo]
f_acid = data['Fa_FSU'][varNo]
chla_ugL = data['Chl_ugL'][varNo]
pheo_ugL = data['Phaeo_ugL'][varNo]

# Compute sigmas
sigma_A = 1/A * np.sqrt(((sigma_phi_chla/phi_chla)**2) + ((sigma_phi_pheo/phi_pheo)**2))
sigma_AoverAminus1 = 1/ (A/(A-1)) * np.sqrt (((sigma_A/A)**2) + ((sigma_A/A)**2))
sigma_Finit_minus_Ffinal = np.sqrt (((sigma_F_init)**2) + ((sigma_F_final)**2))
sigma_vV = sigma_v/sigma_V
sigma_chla = 1/chla_ugL * np.sqrt (((sigma_K/K)**2) + (sigma_AoverAminus1/ (A/(A-1))**2)
                                         + (sigma_Finit_minus_Ffinal/(f_init-f_acid)**2) 
                                         + ((sigma_vV/(v/V))**2))
data['sigma_chla'] = sigma_chla
